**Author:** _Xu Ming_

**Last update:** _2024.04.22_

In [1]:
#导入所需的包
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from data_processing_program_20240419 import deal_special_brand, pre_processing, data_preperation, matching_program, search_regex, search_capacity, mark_unknown_model_with_exsisted_lifting_capacity, mark_outliers, convert_usd_to_cny, update_regex_df, check_col_names, define_load_interval, define_excavator_load_type_interval, check_col_names, matching_program_individual
pd.options.display.float_format = '{:.1f}'.format
import re

# 数据预处理

In [2]:
# 读取dataframe，把文件路径复制到下面代码中
df = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/842952印尼进口-202401-202403.xls', sheet_name="Sheet1")

请确保 **'日期', '产品描述', '供应商', '美元金额', '重量', '数量', '单位'** 等字段在表格的表头里

In [5]:
df.columns = ['月度', '海关编码', '产品描述', '编码描述', '品牌', '进口商', '供应商', '原产国', '原产国所在州',
       '数量', '单位', '金额美元', '美元单价', '合同金额', '合同单价', '日期', '关单号', '进出口',
       '印尼目的港', '国外装货港', '进口商地址', '出口商地址', '币种', '金额印尼盾', '单价印尼盾', '美元重量计单价',
       '重量', '公吨']

把月度列转换成**日期格式**

In [10]:
df['日期'] = pd.to_datetime(df['日期'])

In [18]:
df = df[['月度', '海关编码', '产品描述', '进口商', '供应商', '原产国', '原产国所在州',
       '数量', '单位', '金额美元', '美元单价', '合同金额', '合同单价', '日期', '关单号', '进出口',
       '印尼目的港', '国外装货港', '进口商地址', '出口商地址', '币种', '金额印尼盾', '单价印尼盾',
       '重量', '公吨']]

检查单位，确保单位统一。对于不统一的单位要单独检查并统一，例如：表格中存在unit和set

In [11]:
df['单位'].value_counts()

Number of international units    1107
Set                                97
Piece                              14
pack                                1
Name: 单位, dtype: int64

In [ ]:
df.loc[df['单位']=='Set','数量'].value_counts()

1     86
2      4
26     1
4      1
21     1
6      1
3      1
20     1
12     1
Name: 数量, dtype: int64

In [31]:
df.loc[(df['单位']=='Set')&(df['数量']>5),['产品描述','数量','美元单价','金额美元','重量','供应商']]

,产品描述,数量,美元单价,金额美元,重量,供应商
488,EXCAVATOR,26,117886.8,3065058.0,998894.0,ZHENSHI HOLDING GROUP CO LTD
604,EXCAVATOR,21,81494.2,1711379.0,460950.0,ZHENSHI HOLDING GROUP CO LTD
609,COMPLETE SET EXCAVATOR,6,16294.9,97769.3,0.0,SINOAUTO INTERNATIONAL TRADE (SHANDONG)
720,EXCAVATOR,20,58675.4,1173508.0,336800.0,ZHENSHI HOLDING GROUP CO LTD
890,EXCAVATOR,12,15416.4,184997.3,NaN,SINOAUTO INTERNATIONAL TRADE (SHANDONG)


In [36]:
df.loc[df['单位']=='Set','单位']='UNT'

In [38]:
df.loc[df['单位']=='Piece','数量'].value_counts()

1     7
2     2
4     2
3     1
5     1
10    1
Name: 数量, dtype: int64

In [39]:
df.loc[(df['单位']=='Piece')&(df['数量']>2),['产品描述','数量','美元单价','金额美元','重量','供应商']]

,产品描述,数量,美元单价,金额美元,重量,供应商
540,CRAWLER EXCAVATOR HW60,3,18830.0,56490.0,17430.0,JINING FANGZE MACHINERY CO LTD
545,CRAWLER EXCAVATOR HW35,4,16700.0,66800.0,10600.0,JINING FANGZE MACHINERY CO LTD
689,CRAWLER EXCAVATOR HW25,5,9300.0,46500.0,10449.0,JINING FANGZE MACHINERY CO LTD
715,CRAWLER EXCAVATOR HW10,10,3500.0,35000.0,8726.0,JINING FANGZE MACHINERY CO LTD
716,CRAWLER EXCAVATOR HW17,4,6200.0,24800.0,4524.0,JINING FANGZE MACHINERY CO LTD


In [40]:
df.loc[df['单位']=='Piece','单位']='UNT'

In [41]:
df.loc[df['单位']=='pack','数量'].value_counts()

4    1
Name: 数量, dtype: int64

In [42]:
df.loc[(df['单位']=='pack'),['产品描述','数量','美元单价','金额美元','重量','供应商']]

,产品描述,数量,美元单价,金额美元,重量,供应商
502,1 UNIT OF ERE60 CRAWLER EXCAVATOR COMPLETE WIT...,4,6309.3,25237.3,0.0,QINGDAO EVERUN MACHINERY CO LTD


In [44]:
df.loc[502,['数量','单位']] = [1,'UNT']

In [46]:
df['单位'] = 'UNT'

计算条目单价和净重（以吨为单位）

$$unit\ price = \frac{amount}{quantity}$$

$$weight\ (ton) = \frac{weight\ (kg)}{quantity \times 1000}$$

In [50]:
df.columns = ['月度', '海关编码', '产品描述', '进口商', '供应商', '原产国', '原产国所在州', '数量', '单位', '美元金额',
       '美元单价', '合同金额', '合同单价', '日期', '关单号', '进出口', '印尼目的港', '国外装货港', '进口商地址',
       '出口商地址', '币种', '金额印尼盾', '单价印尼盾', '重量', '公吨']

In [51]:
# 将数量和重量转换为数字格式便于计算
df['数量'] = pd.to_numeric(df['数量'], errors='coerce')
df['重量'] = pd.to_numeric(df['重量'], errors='coerce')

# 计算单价和单位数量吨净重
df['美元单价'] = df['美元金额']/df['数量']
df['单位数量吨净重'] = df['重量']/df['数量']/1000

**根据实际情况需要做相应调整，此份数据有很多微挖和旧机器，此次不做删除**
* 剔除总金额在10000美金以下的条目
* 删除单价在10000美金以下的条目
* 删除重量在5吨以下的条目

In [65]:
df['美元单价'].sort_values(ascending=True)

938      1650.0
991      1680.0
419      1680.0
772      1680.0
511      1680.0
         ...   
692   2020497.7
932   2028211.4
196   3025420.0
200   3025420.0
659   3025421.0
Name: 美元单价, Length: 1219, dtype: float64

In [ ]:
df.loc[df['美元单价']<10000,['产品描述','美元单价']].value_counts()

产品描述                                                                             美元单价  
USED EXCAVATOR WITH ACCESSORIES                                                  2610.0    11
USED EXCAVATOR MACHINE AND PARTS WITH ACCESSORIES                                7000.0     5
USED EXCAVATOR WITH ACCESSORIES                                                  2700.0     4
USED EXCAVATOR C/W PARTS AND ACCESSORIES                                         1990.0     3
USED EXCAVATOR WITH ACCESSORIES                                                  2110.0     3
                                                                                           ..
USED EXCAVATOR AND PARTS S/N : HHEAYH00A00111659 HITACHI ZAXIS ZX70 YEAR 2020    2700.0     1
USED EXCAVATOR AND PARTS S/N : HHEAYE00J00105166 HITACHI ZAXIS ZX60 YEAR 2017    2000.0     1
USED EXCAVATOR AND PARTS S/N : HHEAYE00C00103413 HITACHI ZAXIS ZX60 YEAR 2016    2200.0     1
USED EXCAVATOR AND PARTS S/N : HCM1CG00U00042508 HITACHI ZAXIS ZX7

In [ ]:
# # 先剔除总金额在10000美金以下的条目
# df = df[~(df['美元金额']<10000)]

# # 删除单价在10000美金以下的条目
# df = df[df['美元单价']>=10000]
# df = df.reset_index(drop=True)

# # 删除重量在5吨以下的条目
# df = df[df['单位数量吨净重']>=5]

In [71]:
# 加载品牌型号对照表
df_ref = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/数据对照表/型号对照/20240115_土方对照表_含吨位和马力.xlsx', sheet_name="对照表")

**data_preperation(df, df_ref)定义**

* 输入参数
    * 原始数据表
    * 数据对照表
* 输出
    * 新建**产品描述**副本以免影响原始数据；
    * 去除描述中存在的特殊字符及空格

In [72]:
# 数据预处理：新建产品描述副本以免影响原始数据，去除描述中存在的特殊字符
data_preperation(df, df_ref)

# 对照表匹配

目的：将对照表里存在的设备信息匹配到目前正在处理的数据中

**matching_program()定义:**
* 输入参数（依次）：原始数据表，对照表，文件类型。其中文件类型可填写 'crane' 或 'excavator'
* 输出：匹配后的表格；报错的行号（若有）

**运行机制及标准：**

**1. 将数据对照表里存在的信息对应关系匹配到原始数据里**
* 对照表里的品牌是否存在于产品描述里 -> 存在标记品牌；备注：描述中存在品牌；存在多个匹配品牌时，取第一个品牌
* 对照表里的品牌是否存在于供应商描述里 -> 存在标记品牌；备注：供应商存在品牌；存在多个匹配品牌时，取第一个品牌
* 存在品牌的条目 -> 对照表筛选该品牌 -> 标记匹配的型号；型号唯一时；备注：完美匹配
* 存在品牌的条目 -> 对照表筛选该品牌 -> 标记匹配的型号；有多个型号匹配时，取最长的型号；备注：多个型号匹配，取最长的型号
* 没有匹配的品牌 -> 将在下一个步骤通过通用表达式对条目进行搜索
* 什么都没有的时候 -> 备注：无品牌型号匹配

**2. 删除包含不相关关键词的条目**

    目前统计不相关关键词含如下：

    ['CARRIER', 'TELESCOPLADER', 'HARBOUR', 'OPEN SHEET', 'STACK', 'BOAT', 'BACKHOE', 'SKID', 'ROLLER', 'BENZ', 'TELEHANDLER', 'LOADER', 'FORK', 'PAVER', 'STACKER', 'MATERIAL HANDLER', 'BRIDGE', 'REACH', 'HANDER', 'GRABBER', GANTRY', 'BACK HOE', 'PORT', 'MERCEDES', 'VİNCE', 'SPIDER', 'PIPE', 'HANDLING', 'GLASS CRANE', 'LOAD', 'GRADER', 'GLASS CRANE', 'SPIDER']
                            
**3. 根据货物描述包含的关键词判断产品类型**
* 挖掘机：
    * crawler excavator -> EXCAVATOR
    * wheel 或 tire -> WHEEL EXCAVATOR
    * AMPHIBIOUS -> AMPHIBIOUS EXCAVATOR
    * 不包含上述关键词的 -> UNKNOWN
* 工起：
    * Wheel 或 tire -> WHEELED CRANE
    * rough -> ROUGH-TERRAIN CRANE
    * crawler -> CRAWLER CRANE
    * crawler 和 telescopic -> CRAWLER CRANE (TELESCOPIC_BOOM)
    * 不包含上述关键词的 -> UNKNOWN
    
**4. 判断新旧机**
* 包含如下关键词：['USED', 'SECOND HAND', 'SECONDHAND', 'OLD', '2ND HAND', 'REFURBISH']
* 包含数字：1950 至 2019

    **标记为旧机**

**5. 零部件**
* 包含如下关键词：['CKD', 'SKD', 'partial'] 备注为 **‘散装件’**

In [73]:
# 输入参数：原始数据表，对照表，文件类型，其中文件类型可填写 'crane' 或 'excavator'
# 输出：匹配后的表格；报错的行号（若有）
df, error_rows_index = matching_program(df, df_ref, file_type='excavator')

In [74]:
# 检查是否有error项
error_rows_index.keys()

dict_keys([])

In [75]:
# 查看备注统计
df['备注'].value_counts()

完全匹配           814
无品牌匹配          197
存在品牌，没有匹配型号    168
散装件             16
Name: 备注, dtype: int64

# 通用表达式匹配
目的：根据各品牌的型号命名规则，对上一步没有匹配上的数据进行规则匹配并进行信息标记

In [77]:
# 读取通用表达式对照表
df_regex = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/数据对照表/Regex/20240312_土方REGEX对照表.xlsx',sheet_name='对照表')

**search_regex() 输入参数（依次）**
* 原始表格
* 筛选条件
* 搜索列的列名
* 通用表达式对照表
* 是否对品牌进行筛选 **filter_brand=True/False**
    * True: 如果条目已被标记过品牌，将**只会对该品牌下的**通用表达式规律进行搜索匹配
    * False: 将会忽视品牌，根据**所有**对照表里的通用表达式规律进行搜索匹配
* 是否标记吨位/吊重 **mark_load=True/False**
    * True: mark
    * False: ignore

In [79]:
# 输入参数：原始表格，筛选条件，搜索列名，通用表达式对照表，选择是否通过品牌进行筛选
# 对'存在品牌，没有匹配型号','无品牌匹配'
df = search_regex(df, df['备注'].isin(['存在品牌，没有匹配型号']), '货物描述', df_regex, filter_brand=True, mark_load=True)

In [80]:
df = search_regex(df, df['备注'].isin(['无品牌匹配']), '货物描述', df_regex, filter_brand=False, mark_load=True)

**search_capacity() 输入参数（依次）**
* 原始表格
* 搜索条件
* 搜索列名
* 搜索规律通用表达式

目的：检索描述中包含 **$数字 + (metric) \ TONS$** 关键词。这部分将会把TONS前面的数字标记为吊重。

这部分的数据会被备注为：_描述中包含吊装载重能力描述_

In [95]:
# 只搜索吨位标记为未知，但描述中包含tons的条目
condition = df['吨位']=='UNKNOWN'
search_col = '货物描述'
capacity_regex = r'\b(\d+(?:\.\d+)?)\s*(?:METRIC\s*)?TONS?\b'
# (?:METRIC\s*)?: Matches an optional 'METRIC' followed by optional whitespace characters. 
# The (?: ... ) is a non-capturing group.

df = search_capacity(df, condition, search_col, capacity_regex)

In [143]:
df.loc[df['备注'].isin(['存在品牌，没有匹配型号']),'品牌'].unique()

array(['CATERPILLAR', 'XGMA', 'KOMATSU', 'DOOSAN', 'HITACHI', 'HYUNDAI',
       'LOVOL', 'MAXPOWER', 'ZOOMLION', 'LONKING', 'LIEBHERR'],
      dtype=object)

In [227]:
df['备注'].value_counts()

完全匹配                                    814
无品牌匹配                                   164
根据规律匹配，型号唯一                             124
存在品牌，没有匹配型号                              31
描述中未找到对应品牌，根据规律匹配型号，型号唯一                 29
散装件                                      16
根据规律，有多个匹配结果，但只保留最长型号                    14
描述中包含吊装载重能力描述                             2
描述中未找到对应品牌，根据规律匹配型号，有多个匹配结果，但只保留最长型号      1
Name: 备注, dtype: int64

Crawler crane 300T以上的基本可以判断为 Lattice boom

In [249]:
# 将吨位转换为数字以便计算
df['吨位'] = pd.to_numeric(df['吨位'], errors='coerce')
# df.loc[(df['类型'].isin(['CRAWLER CRANE']))&(df['吨位'].notna())&(df['吨位']>300),'类型'] = 'CRAWLER CRANE (LATTICE_BOOM)'

In [278]:
 df.loc[df['品牌']=='UNIMAX',['型号','货物描述','供应商','备注']]

,型号,货物描述,供应商,备注
497,E69W,WHEEL EXCAVATOR BE69W-B,SHANDONG KEN STONE HEAVY MACHINERY CO,描述中未找到对应品牌，根据规律匹配型号，型号唯一


In [269]:
update_list = [{'brand': 'BOMAC', 'model_regex': r'BE\d{2,3}-d?[BCLXY]', 'capcacity_regex':r'BE(\d+)', 'category':'WHEEL EXCAVATOR', 'starting_point':0}]
df_regex = update_regex_df(df_regex, update_list)

In [295]:
df = search_regex(df, df['货物描述']=='WHEEL EXCAVATOR BE69W-B', '货物描述', df_regex, filter_brand=True, mark_load=True)

In [296]:
df.loc[df['货物描述']=='WHEEL EXCAVATOR BE69W-B']

,月度,海关编码,产品描述,进口商,供应商,原产国,原产国所在州,数量,单位,美元金额,...,重量,公吨,单位数量吨净重,货物描述,品牌,型号,吨位,类型,备注,新旧
497,202402,84295200,WHEEL EXCAVATOR BE69W-B,PT INDOTARA PERSADA,SHANDONG KEN STONE HEAVY MACHINERY CO,China (CN),Asia,2,UNT,51647.5,...,13980.0,14.0,7.0,WHEEL EXCAVATOR BE69W-B,BOMAC,BE69W-B,6.9,WHEEL EXCAVATOR,根据规律匹配，型号唯一,新


因为CATERPILLAR的型号命名规则简单，很容易受主机序列号影响被错误标记，因此单独处理

In [302]:
df.loc[df['品牌']=='CATERPILLAR',['型号','类型','新旧','备注']].value_counts()

型号       类型               新旧  备注         
305.5E2  EXCAVATOR        新   完全匹配           12
320      EXCAVATOR        新   完全匹配            7
330 GC   EXCAVATOR        新   完全匹配            7
UNKNOWN  UNKNOWN          旧   存在品牌，没有匹配型号     7
313      EXCAVATOR        新   完全匹配            6
305      EXCAVATOR        旧   完全匹配            5
307E     EXCAVATOR        旧   完全匹配            4
305.5E2  EXCAVATOR        旧   完全匹配            3
307      EXCAVATOR        新   完全匹配            3
6020B    EXCAVATOR        新   完全匹配            3
307      EXCAVATOR        旧   完全匹配            2
345 GC   EXCAVATOR        新   完全匹配            2
211      EXCAVATOR        旧   完全匹配            1
333      EXCAVATOR        新   完全匹配            1
M320     WHEEL EXCAVATOR  新   完全匹配            1
dtype: int64

In [305]:
cat_regex = pd.read_excel('数据对照表/Regex/20240312_土方CAT_REGEX对照表.xlsx',sheet_name='对照表')

In [311]:
cat_regex.columns = ['brand', 'model_regex', 'capacity_regex', 'category', 'starting_point']

In [312]:
df = search_regex(df, (df['品牌']=='CATERPILLAR')&(df['型号']=='UNKNOWN'), '货物描述', cat_regex, filter_brand=True, mark_load=True)

In [316]:
df['备注'].value_counts()

完全匹配                                    814
无品牌匹配                                   164
根据规律匹配，型号唯一                             125
描述中未找到对应品牌，根据规律匹配型号，型号唯一                 28
存在品牌，没有匹配型号                              24
根据规律，有多个匹配结果，但只保留最长型号                    21
散装件                                      16
描述中包含吊装载重能力描述                             2
描述中未找到对应品牌，根据规律匹配型号，有多个匹配结果，但只保留最长型号      1
Name: 备注, dtype: int64

# 利用已知信息反向标记
此部分将用已标记了品牌、型号、吊重、类型信息的条目（A条目），反向匹配包含品牌、吊重和类型的条目（B条目）。

运行条件：只要满足品牌和类型匹配，且B的吊重在已知A吊重的±5%范围内。

例如：已知A和B两个条目品牌和类型一致，A包含上述的4个已知信息。B的吊重通过数字+TONS的方式知道该信息。下面的方程将会计算B的吨位±5%作为阈值，如果A的吊重正好在这个区间内，那判断B的型号和A一致。

| 条目 | 品牌 | 型号 | 吊重 | 类型 |
|------|------|------|------|------|
| A    | ✓    | ✓    | ✓    | ✓    |
| B    | ✓    | ✘    | ✓    | ✓    |

In [333]:
# 参数只需要填写需要进行标记的数据表
df = mark_unknown_model_with_exsisted_lifting_capacity(df)

In [334]:
# 计算无品牌匹配条目的金额和数量
unlabeled_value = round(df.loc[df['备注'] == '无品牌匹配','美元金额'].sum(),1)
unlabeled_qty = df.loc[df['备注'] == '无品牌匹配','数量'].sum()
print(f'无品牌匹配条目总计金额: {round(unlabeled_value/10000,2)} 万美金')
print(f'无品牌匹配条目总计数量: {unlabeled_qty} 台')

无品牌匹配条目总计金额: 941.45 万美金
无品牌匹配条目总计数量: 331 台


# 标记异常值

检查是否有空值

In [335]:
df['单位数量吨净重'].isna().sum()

285

**mark_outliers(df, term=True)**
* 输入参数（依次）：
    * 原始数据表
    * 条件：可选择 **True/False**；另：如果存在条款列，term参数需填写为True，另外要确保该条款列叫 “交易条款”
        * True：异常值判断将以品牌、型号、条款为筛选条件，取上述三个标记同时满足情况下的**价格中位数**为参照标准；
        * False：如果原始表格不包含条款，选择false将不会以条款为筛选条件，并**只会对同品牌型号下的价格取中位数**按照上述标准进行判断。

**目的：** 找到价格异常高和异常低的条目，以便后续进行价格分析

**价格异常判断标准：**

计算同品牌型号单价的的中位数，如果存在贸易条款，要同一贸易条款下的进行判断
* 未知：型号未知、旧机；
* 正常：条目的价格在中位数±20%内，标记为不异常；
* 异常：上述阈值范围外为价格异常。

In [340]:
df = mark_outliers(df, term=False)

# 吨位或吊重区间划分
**define_load_interval(df, file_type, load_interval)**
输入参数（依次）
* 原始数据表
* 文件类型：需要填写 **crane** 或 **excavator**；注意：挖掘机和工起的划分逻辑不一样
* 区间划分间隔
   * 若填写50，则以50为一个区间划分。例如：≤50T, 50-100T, 100-150T依次类推；
   * 工起数据写50比较合理

**工起**
* 工起存在吊重远远大于自重的情况，而且不同种类的情况不同，不能通过设备自身重量来进行推测；
* 只会对标记了吊重的条目进行区间划分
    
    
**挖掘机**

挖掘机的吨位和设备本身的自重几乎一致，对未知条目会通过重量进行划分
* 对已知吨位进行划分
* 对未知吨位的条目，通过重量进行划分

In [411]:
define_load_interval(df, file_type = 'excavator', load_interval = 10)

In [441]:
define_excavator_load_type_interval(df)

In [442]:
df['吨位类型划分'].value_counts()

5-10T      726
10-30T     263
30-70T     121
<5T         37
UNKNOWN     27
≥90T        15
70-90T       6
Name: 吨位类型划分, dtype: int64

In [412]:
df['吨位范围'].value_counts()

5-10T       726
20-30T      222
30-40T       62
40-50T       41
10-20T       41
<5T          37
50-60T       17
UNKNOWN      12
110-120T      6
70-80T        5
200-210T      4
230-240T      3
80-90T        1
90-100T       1
100-110T      1
60-70T        1
Name: 吨位范围, dtype: int64

In [444]:
df['吨位类型划分'] = df['吨位类型划分'].fillna('UNKNOWN')
df['吨位范围'] = df['吨位范围'].fillna('UNKNOWN')

# 汇率换算及其它

**convert_usd_to_cny(df, rate_dict)**

输入参数：
* 原始数据表：确保存在 ['日期', '美元单价', '美元金额'] 三列
* 汇率换算表：文件形式是个dictionary，最新的汇率更新后需要手动填进‘rate_dict’，历史数据会按照字典里的日期进行计算对应汇率

In [416]:
rate_dict = {2023: {1: 6.7604, 2: 6.9519, 3: 6.8717, 4: 6.924, 5: 7.0821, 6: 7.2258, 7: 7.1305, 8: 7.1811, 9: 7.1798, 10: 7.1779, 11: 7.1018, 12: 7.0827},
             2024: {1: 7.1039, 2: 7.1036, 3: 7.0950, 4:7.1063}}

df = convert_usd_to_cny(df, rate_dict)

In [418]:
# 清楚型号里的除特殊符号及多余空格
df['型号'] = df['型号'].apply(lambda x: x.replace(' ',''))
df['型号'] = df['型号'].apply(lambda x: x.lstrip())
df['型号'] = df['型号'].apply(lambda x: x.rstrip())

_**下面的代码仅在处理工起数据时使用**_

In [ ]:
# 新建工起一二级中文分类
EN_CN = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/数据对照表/工起类型分类对照表.xlsx',sheet_name='Sheet1')

In [ ]:
df = pd.merge(df, EN_CN, how='left', on='类型')

保存目前清理后的数据

In [419]:
# 保存目前清理后的数据
df.to_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/2024年一季度印尼挖掘机进口数据_已清理和异常判断.xlsx', sheet_name='Sheet1', index=False)

# 更新历史数据

将历史数据和最新数据合并，需要满足如下条件：
* 读取历史数据
* 确认逻辑统一 (本文件里的所有方程都可以重复利用来统一历史数据的逻辑)
* 确认所有合并文件的列名一致（位置不重要但名字一定要一致）

In [420]:
df_hist = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/2022年8-2023年印尼挖掘机进口数据快报.xlsx',sheet_name='7_原始数据')

In [422]:
df_hist['吨位类型划分']

0        5-10T
1        5-10T
2       30-70T
3       30-70T
4       30-70T
         ...  
5327    10-30T
5328    10-30T
5329     5-10T
5330    10-30T
5331     5-10T
Name: 吨位类型划分, Length: 5332, dtype: object

**check_col_names(df.columns, df_hist.columns)**

检查合并的表格列名是否一致，左边是original，右边是reference；只要满足其中一个文件的列名是另一个文件的子集即可

In [452]:
check_col_names(df.columns, df_hist.columns)

Columns in the reference df but not in original: {'马力', '公吨', '美元重量计单价', '净重千克'}


数据合并

In [453]:
df_updated = pd.concat([df_hist, df], axis=0, ignore_index=True)

In [455]:
# 清楚型号里的除特殊符号及多余空格
df_updated['型号'] = df_updated['型号'].apply(lambda x: x.replace(' ',''))
df_updated['型号'] = df_updated['型号'].apply(lambda x: x.lstrip())
df_updated['型号'] = df_updated['型号'].apply(lambda x: x.rstrip())

In [456]:
# 保存合并后的数据
df_updated.to_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/2022年8-2024年3月印尼挖掘机进口数据_已清理及异常判断.xlsx',sheet_name='Sheet1',index=False)